### Install Dependencies

In [3]:
!pip install psycopg2

  Using cached psycopg2-2.8.6.tar.gz (383 kB)
  Created wheel for psycopg2: filename=psycopg2-2.8.6-cp37-cp37m-linux_x86_64.whl size=494996 sha256=45593a34e6aca7e680f86df4d7ce978e0e93d6de2842928c3ff37e7007564e27
  Stored in directory: /home/jovyan/.cache/pip/wheels/25/78/75/9c0323f7e1fb42143cbd2439302beb7850a1034abb961cb281
Successfully built psycopg2


### Import Libraries

In [3]:
import psycopg2
import numpy as np
import pandas as pd
import time
import json


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

"""[Connect to database](https://pynative.com/python-postgresql-tutorial/)"""

'[Connect to database](https://pynative.com/python-postgresql-tutorial/)'

### Code

#### Connect to Database

In [4]:
try:
    connection = psycopg2.connect(user = "postgres",
                                  password = "Welcome01",
                                  host = "35.238.29.211",
                                  port = "5432",
                                  database = "dataproject2")
   
    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")
    
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
    
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'postgres', 'dbname': 'dataproject2', 'host': '35.238.29.211', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 10.15 (Debian 10.15-1.pgdg90+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18+deb9u1) 6.3.0 20170516, 64-bit',) 



#### Obtain houses

In [5]:
cursor.execute("SELECT * FROM casas;")
record = cursor.fetchall()

cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'casas'")
columns_name = cursor.fetchall()

"""Convert array of arrays to single array"""

array_columns_name = np.array(columns_name)
array_columns_name = np.concatenate( array_columns_name, axis=0 )

#print(array_columns_name)

"""Transform result of query to a pandas dataframe"""

df = pd.DataFrame(record, columns=array_columns_name)

df.head()

,id,house_id,city_name,rooms,cost,tweet_id,c_counter
0,1,585864,Bilbao,4,434,1352945082241081346,0


##### Define constants

In [6]:
playa = ["Valencia", "Barcelona", "Ibiza"]
ciudad = ["Madrid", "Barcelona", "Valencia", "Sevilla"]
naturaleza = ["Oviedo", "Bilbao"]
fiesta = ["Ibiza", "Madrid", "Barcelona"]

In [7]:
scoring = {"Madrid": 0, "Barcelona": 0, "Ibiza": 0, "Valencia": 0, "Sevilla": 0, "Oviedo": 0, "Bilbao": 0}

#### Match with people

In [8]:
#def tuFuncion(text_person):

text_person = '{"created_at":"Sat Jan 23 12:43:59 +0000 2021","id":1352960267303460866,"full_text":"My name is Tara Leon, my salary is 137546€ yearly and I am 25 years old. My family are 3 members. These are my hobbies: Beach(8), City(1), Nature(0), Party(2)  #mdaedem","text":"My name is Tara Leon, my salary is 137546€ yearly and I am 25 years old. My family are 3 members. These are my hobb… https://t.co/29g2Rcx1Ev"}'
#TODO: obtain text from Kafka using Spark

json_person = json.loads(text_person)

In [9]:
tweet_text = json_person['full_text']
print(tweet_text)

My name is Tara Leon, my salary is 137546€ yearly and I am 25 years old. My family are 3 members. These are my hobbies: Beach(8), City(1), Nature(0), Party(2)  #mdaedem


In [10]:
split_text = tweet_text.split(" ")
salary = split_text[8][:-1]
members = split_text[19]
beach = split_text[25][-3:-2]
city = split_text[26][-3:-2]
nature = split_text[27][-3:-2]
party = split_text[28][-2:-1]

In [11]:
print(beach)

8


#### Conditionals

Obtener las casas que cumplan el salario

In [12]:
casas_coste = df[df.cost <= ((int(salary)/12)*0.2)]
casas_coste.head()

,id,house_id,city_name,rooms,cost,tweet_id,c_counter
0,1,585864,Bilbao,4,434,1352945082241081346,0


Obtener las casas que cumplan las habitaciones

In [13]:
casas_hab1 = casas_coste.loc[casas_coste.rooms == members]
casas_hab2 = casas_coste.loc[casas_coste.rooms == int(members)+1]

casas_hab = pd.concat([casas_hab1,casas_hab2]).drop_duplicates().reset_index(drop=True)
casas_hab.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,id,house_id,city_name,rooms,cost,tweet_id,c_counter
0,1,585864,Bilbao,4,434,1352945082241081346,0


In [15]:
casas_hab = casas_coste.loc[(casas_coste.rooms == members) | ((casas_coste.rooms.astype(int)) == int(members)+1)]
casas_hab.head()

,id,house_id,city_name,rooms,cost,tweet_id,c_counter
0,1,585864,Bilbao,4,434,1352945082241081346,0


Sistema de puntuaciones de las ciudades en base a las preferencias del cliente

In [16]:
for k in scoring:
    try:
        playa.index(k)
        scoring[k] += int(beach)
    except (Exception) as error :
        print("Not present", error)
        
    try:
        ciudad.index(k)
        scoring[k] += int(city)
    except (Exception) as error :
        print("Not present", error)
        
    try:
        naturaleza.index(k)
        scoring[k] += int(nature)
    except (Exception) as error :
        print("Not present", error)
        
    try:
        fiesta.index(k)
        scoring[k] += int(party)
    except (Exception) as error :
        print("Not present", error)

Not present 'Madrid' is not in list
Not present 'Madrid' is not in list
Not present 'Barcelona' is not in list
Not present 'Ibiza' is not in list
Not present 'Ibiza' is not in list
Not present 'Valencia' is not in list
Not present 'Valencia' is not in list
Not present 'Sevilla' is not in list
Not present 'Sevilla' is not in list
Not present 'Sevilla' is not in list
Not present 'Oviedo' is not in list
Not present 'Oviedo' is not in list
Not present 'Oviedo' is not in list
Not present 'Bilbao' is not in list
Not present 'Bilbao' is not in list
Not present 'Bilbao' is not in list


In [17]:
scoring

{'Madrid': 3,
 'Barcelona': 11,
 'Ibiza': 10,
 'Valencia': 9,
 'Sevilla': 1,
 'Oviedo': 0,
 'Bilbao': 0}

Obtener las casas asignadas a menos de 4 clientes

In [18]:
casas_libres = casas_hab.loc[casas_hab.c_counter <= 4]
casas_libres.head()

,id,house_id,city_name,rooms,cost,tweet_id,c_counter
0,1,585864,Bilbao,4,434,1352945082241081346,0


Obtener las casas con mejor puntuación según el scoring anterior

In [20]:
mejor_casa = casas_libres
mejor_casa['score'] = mejor_casa['city_name'].map(scoring)
mejor_casa.head()

,id,house_id,city_name,rooms,cost,tweet_id,c_counter,score
0,1,585864,Bilbao,4,434,1352945082241081346,0,0


In [24]:
resultado = mejor_casa[mejor_casa.score == max(mejor_casa.score)]
resultado.head()

,id,house_id,city_name,rooms,cost,tweet_id,c_counter,score
0,1,585864,Bilbao,4,434,1352945082241081346,0,0
